In [1]:
"""Binary for training translation models and decoding from them.

See the following papers for more information on neural translation models.
 * http://arxiv.org/abs/1409.3215
 * http://arxiv.org/abs/1409.0473
 * http://arxiv.org/abs/1412.2007
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import random
import sys
import time

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.models.rnn.translate import data_utils
from tensorflow.models.rnn.translate import seq2seq_model


tf.app.flags.DEFINE_float("learning_rate", 0.5, "Learning rate.")
tf.app.flags.DEFINE_float("learning_rate_decay_factor", 0.99,
                          "Learning rate decays by this much.")
tf.app.flags.DEFINE_float("max_gradient_norm", 5.0,
                          "Clip gradients to this norm.")
tf.app.flags.DEFINE_integer("batch_size", 64,
                            "Batch size to use during training.")
tf.app.flags.DEFINE_integer("size", 1024, "Size of each model layer.")
tf.app.flags.DEFINE_integer("num_layers", 3, "Number of layers in the model.")
tf.app.flags.DEFINE_integer("en_vocab_size", 40000, "English vocabulary size.")
tf.app.flags.DEFINE_integer("fr_vocab_size", 40000, "French vocabulary size.")
tf.app.flags.DEFINE_string("data_dir", "/tmp", "Data directory")
tf.app.flags.DEFINE_string("train_dir", "/tmp", "Training directory.")
tf.app.flags.DEFINE_integer("max_train_data_size", 0,
                            "Limit on the size of training data (0: no limit).")
tf.app.flags.DEFINE_integer("steps_per_checkpoint", 200,
                            "How many training steps to do per checkpoint.")
tf.app.flags.DEFINE_boolean("decode", False,
                            "Set to True for interactive decoding.")
tf.app.flags.DEFINE_boolean("self_test", False,
                            "Run a self-test if this is set to True.")

FLAGS = tf.app.flags.FLAGS

# We use a number of buckets and pad to the closest one for efficiency.
# See seq2seq_model.Seq2SeqModel for details of how they work.
_buckets = [(5, 10), (10, 15), (20, 25), (40, 50)]



In [32]:
import string

vocabulary_size = len(string.ascii_lowercase) # [a-z]
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter
    else:
        print('Unexpected character: %s' % char)
        return 0
  
def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character:  
Unexpected character: ï
0 25 0 0
b {  


In [7]:
def get_batch(size):
    encoder_inputs = [np.random.randint(1, vocabulary_size, size) for _ in xrange(batch_size)]
    decoder_inputs = [np.zeros(size+1, dtype = np.int32) for _ in xrange(batch_size)]
    weights = [np.ones(size+1, dtype = np.float32) for _ in xrange(batch_size)]
    for i in xrange(batch_size):
        r = random.randint(1, size-1)
        encoder_inputs[i][r:] = 0
        # Reverse the encoder input sequence, but leave a 0 at index 0 and at least one 0 at the end.
        # These are the GO and EOS markers.
        decoder_inputs[i][1:r+1] = encoder_inputs[i][:r][::-1]
        weights[i][r+1:] = 0.0
    return np.transpose(encoder_inputs), np.transpose(decoder_inputs), np.transpose(weights)


Bacthes are three matrices, encoder_inputs and decoder_inputs and weights. Weights are all zeroes.

encoder_inputs have one entry per column (a fake word) with one letter per row.

decoder_inputs (the "labels") are encoder_inputs reversed.

In [28]:
batch_size=20
vocabulary_size=22
b = get_batch(10)
print('e_i')
print(b[0])
print('d_i')
print(b[1])



e_i
[[ 4 10  5 19 10  6  1 18 15 13 14 10  9 20 18 20  1  6 17 21]
 [17 17 10  9 18 12  9  3  2  5  1 10 20  6  5  7  7  1  8  8]
 [20 14 21 11 20 18 10 12  8  5 13 13  2 16  5  0 19  4  0  8]
 [17 20 12  0 13 20  4  7  8  0  0 17  3 11  7  0  3  0  0  2]
 [ 0  1 11  0 13  6 11 12 15  0  0  9 16  2 16  0 12  0  0  7]
 [ 0 21 17  0 15 10 16  5  3  0  0 17  0 19  3  0  9  0  0  1]
 [ 0 14  0  0  5  0 21 14  8  0  0 21  0  5 17  0  0  0  0  0]
 [ 0  0  0  0 19  0  2 15  0  0  0  2  0 20 10  0  0  0  0  0]
 [ 0  0  0  0 17  0  6  7  0  0  0  0  0  8  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
d_i
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [17 14 17 11 17 10  6  7  8  5 13  2 16  8 10  7  9  4  8  1]
 [20 21 11  9 19  6  2 15  3  5  1 21  3 20 17 20 12  1 17  7]
 [17  1 12 19  5 20 21 14 15 13 14 17  2  5  3  0  3  6  0  2]
 [ 4 20 21  0 15 18 16  5  8  0  0  9 20 19 16  0 19  0  0  8]
 [ 0 14 10  0 13 12 11 12  8  0  0 17  9  2  7

In [33]:

def get_validation_batch(text, size):
    words = text.split()[:batch_size]
    encoder_inputs = [np.zeros(size, dtype = np.int32) for _ in xrange(batch_size)]
    for i, word in enumerate(words):
        l = min(len(word), size - 1)
        for j in xrange(l):
            encoder_inputs[i][j] = char2id(word[j])
    return np.transpose(encoder_inputs)

val_encoder_inputs = get_validation_batch("the quick brown fox jumps over the lazy dog", 10)
print(val_encoder_inputs)

[[19 16  1  5  9 14 19 11  3  0  0  0  0  0  0  0  0  0  0  0]
 [ 7 20 17 14 20 21  7  0 14  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  8 14 23 12  4  4 25  6  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2 22  0 15 17  0 24  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 10 13  0 18  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


----------------------

Model
-----



In [29]:
  # model = seq2seq_model.Seq2SeqModel(source_vocab_size = vocabulary_size,
  #                                    target_vocab_size = vocabulary_size,
  #                                     buckets = [(10, 11)],
  #                                     size = num_nodes,
  #                                     num_layers = 1, # one encoding and one decoding LSTM
  #                                     max_gradient_norm = 5.0,
  #                                     batch_size = batch_size,
  #                                     learning_rate = 1.0,
  #                                     learning_rate_decay_factor = 0.9,
  #                                     use_lstm = True,
  #                                     forward_only = False)
    
def create_model(session, forward_only):
  """Create translation model and initialize or load parameters in session."""
  model = seq2seq_model.Seq2SeqModel(
      FLAGS.en_vocab_size, FLAGS.fr_vocab_size, _buckets,
      FLAGS.size, FLAGS.num_layers, FLAGS.max_gradient_norm, FLAGS.batch_size,
      FLAGS.learning_rate, FLAGS.learning_rate_decay_factor,
      forward_only=forward_only)
  ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)
  if ckpt and tf.gfile.Exists(ckpt.model_checkpoint_path):
    print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
    model.saver.restore(session, ckpt.model_checkpoint_path)
  else:
    print("Created model with fresh parameters.")
    session.run(tf.initialize_all_variables())
  return model


In [4]:
def train():
  """Train a en->fr translation model using WMT data."""
  # Prepare WMT data.
  print("Preparing WMT data in %s" % FLAGS.data_dir)
  en_train, fr_train, en_dev, fr_dev, _, _ = data_utils.prepare_wmt_data(
      FLAGS.data_dir, FLAGS.en_vocab_size, FLAGS.fr_vocab_size)

  with tf.Session() as sess:
    # Create model.
    print("Creating %d layers of %d units." % (FLAGS.num_layers, FLAGS.size))
    model = create_model(sess, False)

    # Read data into buckets and compute their sizes.
    print ("Reading development and training data (limit: %d)."
           % FLAGS.max_train_data_size)
    dev_set = read_data(en_dev, fr_dev)
    train_set = read_data(en_train, fr_train, FLAGS.max_train_data_size)
    train_bucket_sizes = [len(train_set[b]) for b in xrange(len(_buckets))]
    train_total_size = float(sum(train_bucket_sizes))

    # A bucket scale is a list of increasing numbers from 0 to 1 that we'll use
    # to select a bucket. Length of [scale[i], scale[i+1]] is proportional to
    # the size if i-th training bucket, as used later.
    train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                           for i in xrange(len(train_bucket_sizes))]

    # This is the training loop.
    step_time, loss = 0.0, 0.0
    current_step = 0
    previous_losses = []
    while True:
      # Choose a bucket according to data distribution. We pick a random number
      # in [0, 1] and use the corresponding interval in train_buckets_scale.
      random_number_01 = np.random.random_sample()
      bucket_id = min([i for i in xrange(len(train_buckets_scale))
                       if train_buckets_scale[i] > random_number_01])

      # Get a batch and make a step.
      start_time = time.time()
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          train_set, bucket_id)
      _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs,
                                   target_weights, bucket_id, False)
      step_time += (time.time() - start_time) / FLAGS.steps_per_checkpoint
      loss += step_loss / FLAGS.steps_per_checkpoint
      current_step += 1

      # Once in a while, we save checkpoint, print statistics, and run evals.
      if current_step % FLAGS.steps_per_checkpoint == 0:
        # Print statistics for the previous epoch.
        perplexity = math.exp(loss) if loss < 300 else float('inf')
        print ("global step %d learning rate %.4f step-time %.2f perplexity "
               "%.2f" % (model.global_step.eval(), model.learning_rate.eval(),
                         step_time, perplexity))
        # Decrease learning rate if no improvement was seen over last 3 times.
        if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
          sess.run(model.learning_rate_decay_op)
        previous_losses.append(loss)
        # Save checkpoint and zero timer and loss.
        checkpoint_path = os.path.join(FLAGS.train_dir, "translate.ckpt")
        model.saver.save(sess, checkpoint_path, global_step=model.global_step)
        step_time, loss = 0.0, 0.0
        # Run evals on development set and print their perplexity.
        for bucket_id in xrange(len(_buckets)):
          if len(dev_set[bucket_id]) == 0:
            print("  eval: empty bucket %d" % (bucket_id))
            continue
          encoder_inputs, decoder_inputs, target_weights = model.get_batch(
              dev_set, bucket_id)
          _, eval_loss, _ = model.step(sess, encoder_inputs, decoder_inputs,
                                       target_weights, bucket_id, True)
          eval_ppx = math.exp(eval_loss) if eval_loss < 300 else float('inf')
          print("  eval: bucket %d perplexity %.2f" % (bucket_id, eval_ppx))
        sys.stdout.flush()


In [34]:
# should study this.

def evaluate_model(model, sess, val_encoder_inputs, output_size):
    val_decoder_inputs = np.zeros((output_size+1, batch_size,), dtype = np.int32)
    val_target_weights = np.zeros((output_size+1, batch_size,), dtype = np.float32)
    val_target_weights[0,:] = 1.0
    is_finished = np.full((batch_size,), False, dtype = np.bool_)
    for i in xrange(output_size):
        _, _, output_logits = model.step(sess, val_encoder_inputs, val_decoder_inputs, val_target_weights, bucket_id = 0, forward_only = True)
        p = np.argmax(output_logits[i], axis = 1)
        is_finished = np.logical_or(is_finished, p == 0)
        val_decoder_inputs[i,:] = (1 - is_finished) * p
        val_target_weights[i,:] = (1.0 - is_finished) * 1.0
    return val_decoder_inputs

In [5]:
def decode():
  with tf.Session() as sess:
    # Create model and load parameters.
    model = create_model(sess, True)
    model.batch_size = 1  # We decode one sentence at a time.

    # Load vocabularies.
    en_vocab_path = os.path.join(FLAGS.data_dir,
                                 "vocab%d.en" % FLAGS.en_vocab_size)
    fr_vocab_path = os.path.join(FLAGS.data_dir,
                                 "vocab%d.fr" % FLAGS.fr_vocab_size)
    en_vocab, _ = data_utils.initialize_vocabulary(en_vocab_path)
    _, rev_fr_vocab = data_utils.initialize_vocabulary(fr_vocab_path)

    # Decode from standard input.
    sys.stdout.write("> ")
    sys.stdout.flush()
    sentence = sys.stdin.readline()
    while sentence:
      # Get token-ids for the input sentence.
      token_ids = data_utils.sentence_to_token_ids(tf.compat.as_bytes(sentence), en_vocab)
      # Which bucket does it belong to?
      bucket_id = min([b for b in xrange(len(_buckets))
                       if _buckets[b][0] > len(token_ids)])
      # Get a 1-element batch to feed the sentence to the model.
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          {bucket_id: [(token_ids, [])]}, bucket_id)
      # Get output logits for the sentence.
      _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs,
                                       target_weights, bucket_id, True)
      # This is a greedy decoder - outputs are just argmaxes of output_logits.
      outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
      # If there is an EOS symbol in outputs, cut them at that point.
      if data_utils.EOS_ID in outputs:
        outputs = outputs[:outputs.index(data_utils.EOS_ID)]
      # Print out French sentence corresponding to outputs.
      print(" ".join([tf.compat.as_str(rev_fr_vocab[output]) for output in outputs]))
      print("> ", end="")
      sys.stdout.flush()
      sentence = sys.stdin.readline()



In [6]:
def self_test():
  """Test the translation model."""
  with tf.Session() as sess:
    print("Self-test for neural translation model.")
    # Create model with vocabularies of 10, 2 small buckets, 2 layers of 32.
    model = seq2seq_model.Seq2SeqModel(10, 10, [(3, 3), (6, 6)], 32, 2,
                                       5.0, 32, 0.3, 0.99, num_samples=8)
    sess.run(tf.initialize_all_variables())

    # Fake data set for both the (3, 3) and (6, 6) bucket.
    data_set = ([([1, 1], [2, 2]), ([3, 3], [4]), ([5], [6])],
                [([1, 1, 1, 1, 1], [2, 2, 2, 2, 2]), ([3, 3, 3], [5, 6])])
    for _ in xrange(5):  # Train the fake model for 5 steps.
      bucket_id = random.choice([0, 1])
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          data_set, bucket_id)
      model.step(sess, encoder_inputs, decoder_inputs, target_weights,
                 bucket_id, False)
